<a href="https://colab.research.google.com/github/mz-zarei/Trajectory_Analysis/blob/main/INTERACTION_DATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [3]:
folder = '/content/drive/MyDrive/Trajectory_Analysis/Interaction/'
data_paths_train = [folder + i for i in os.listdir(folder)]

num_cases = 0
max_num_agents = 0
for path in data_paths_train:
    print(path)
    data = pd.read_csv(path)
    #droping unused columns
    # data.drop(['vx','vy','psi_rad','length','width'], axis=1, inplace=True)
    # data.to_csv(path, index=False)
    print("\t Number of cases: ", data.case_id.max())
    print("\t Max number of agents in case: ", data.track_id.max())

    num_cases += data.case_id.max()
    
    if max_num_agents < data.track_id.max():
        max_num_agents = data.track_id.max()

print("-" * 60)
print("Total number of cases: ", num_cases)
print("Max number of agents in all cases: ", max_num_agents)

/content/drive/MyDrive/Trajectory_Analysis/Interaction/DR_USA_Roundabout_EP_val.csv
	 Number of cases:  433.0
	 Max number of agents in case:  23
/content/drive/MyDrive/Trajectory_Analysis/Interaction/DR_DEU_Roundabout_OF_val.csv
	 Number of cases:  568.0
	 Max number of agents in case:  10
/content/drive/MyDrive/Trajectory_Analysis/Interaction/DR_CHN_Roundabout_LN_val.csv
	 Number of cases:  238.0
	 Max number of agents in case:  7
/content/drive/MyDrive/Trajectory_Analysis/Interaction/DR_USA_Roundabout_SR_val.csv
	 Number of cases:  552.0
	 Max number of agents in case:  29
/content/drive/MyDrive/Trajectory_Analysis/Interaction/DR_USA_Roundabout_FT_val.csv
	 Number of cases:  2452.0
	 Max number of agents in case:  23
/content/drive/MyDrive/Trajectory_Analysis/Interaction/DR_USA_Roundabout_FT_train.csv
	 Number of cases:  9811.0
	 Max number of agents in case:  25
/content/drive/MyDrive/Trajectory_Analysis/Interaction/DR_CHN_Roundabout_LN_train.csv
	 Number of cases:  1078.0
	 Max nu

In [4]:
def find_max_coordinates(data_paths, rot_deg_increment=360):
    raw_data_buffer = []
    for path in data_paths:
        # print(path)
        # Load data from csv file.
        data = pd.read_csv(path)
        data = data[['x','y']].to_numpy()
        raw_data_buffer.append(data)            
        # Rotate data by deg_increment deg sequentially for data augmentation (only rotation is considered here)
        deg_increment_int = int(rot_deg_increment)
        for deg in range(deg_increment_int, 360, deg_increment_int):
            data_rotated = np.zeros_like(data)
            rad = np.radians(deg)
            c, s = np.cos(rad), np.sin(rad)
            Rot = np.array(((c,-s), (s, c)))
            data_rotated = np.dot(Rot, data.T).T
            raw_data_buffer.append(data_rotated)
    # Find x_max, x_min, y_max, y_min across all the data in data_paths.
    x_max_global, x_min_global, y_max_global, y_min_global = -1000, 1000, -1000, 1000
    for data in raw_data_buffer:
        x = data[:,0]
        x_min, x_max = min(x), max(x)
        if x_min < x_min_global:
            x_min_global = x_min
        if x_max > x_max_global:
            x_max_global = x_max
        y = data[:,1]
        y_min, y_max = min(y), max(y)
        if y_min < y_min_global:
            y_min_global = y_min
        if y_max > y_max_global:
            y_max_global = y_max
    return x_max_global, x_min_global, y_max_global, y_min_global

def normalize(x_values, y_values , x_max_global, x_min_global, y_max_global, y_min_global):
    # Normalize x and y values to range from -1 to 1.
    normalized_x_values = np.zeros(len(x_values))
    normalized_y_values = np.zeros(len(y_values))
    x_normalized = (1 + 1)*(x_values - x_min_global)/(x_max_global - x_min_global)
    x_normalized = x_normalized - 1.0
    for jj in range(len(x_normalized)):
        if abs(x_normalized[jj]) < 0.0001:
            normalized_x_values[jj] = 0.0
        else:
            normalized_x_values[jj] = x_normalized[jj] 
    y_normalized = (1 + 1)*(y_values - y_min_global)/(y_max_global - y_min_global)
    y_normalized = y_normalized - 1.0
    for jj in range(len(y_normalized)):
        if abs(y_normalized[jj]) < 0.0001:
            normalized_y_values[jj] = 0.0
        else:
            normalized_y_values[jj] = y_normalized[jj]
    return normalized_x_values, normalized_y_values

def preprocess(data_paths, rot_deg_increment=360):
    # Find x_max, x_min, y_max, y_min across all the data in data_paths
    x_max_global, x_min_global, y_max_global, y_min_global = find_max_coordinates(data_paths, rot_deg_increment)
    print("x_max, x_min, y_max, y_min across all the data in data_paths: ",
          x_max_global, x_min_global, y_max_global, y_min_global)
    preprocessed_data_all = []

    deg_increment_int = int(rot_deg_increment)
    for path in data_paths:
        print("Preprocessing train data set: ", path)
        data = pd.read_csv(path)
        data = data[data.frame_id % 2 == 0].copy() # skip one frame
        all_cases = data.case_id.astype(int).unique()
        for deg in range(0, 360, deg_increment_int):
            print("Rotation degree: ", deg)
            # Rotate x and y values for the agent
            rad = np.radians(deg)
            c, s = np.cos(rad), np.sin(rad)
            Rot = np.array(((c,-s), (s, c)))
            for case in tqdm(all_cases):
                agent_ids = data[data.case_id == case].track_id.astype(int).unique()
                preprocessed_data = np.zeros((3,max_num_agents,40))                              

                for agent in agent_ids:
                    x = data[(data.case_id == case) & (data.track_id==agent)].x.values
                    y = data[(data.case_id == case) & (data.track_id==agent)].y.values
                    agent_type = np.array([1 if i == 'car' else 0 for i in data[(data.case_id == case) & (data.track_id == agent)].agent_type.values])
                    length = len(x)

                    # Rotate x and y values for the agent
                    xy_rotated = np.dot(Rot, np.array([x,y]))

                    # Normalize rotated xy
                    x_normal_rotated, y_normal_rotated = normalize(xy_rotated[0], xy_rotated[1] , x_max_global, x_min_global, y_max_global, y_min_global)
                    
                    # Add rotated-normalized x, y and agent type to trajectory matrix
                    preprocessed_data[0,agent-1, :length] = x_normal_rotated
                    preprocessed_data[1,agent-1, :length] = y_normal_rotated
                    preprocessed_data[2,agent-1, :length] = agent_type

                preprocessed_data_all.append(preprocessed_data)
    return np.stack(preprocessed_data_all, axis =0)

In [5]:
folder = '/content/drive/MyDrive/Trajectory_Analysis/Interaction/'
data_paths = [folder + i for i in os.listdir(folder)]
rot_deg_increment = 360
preprocessed_data_all = preprocess(data_paths, rot_deg_increment=rot_deg_increment)
np.save(folder + 'preprocessed_data_rot_' + str(360), preprocessed_data_all)

x_max, x_min, y_max, y_min across all the data in data_paths:  1099.1680000000001 916.043 1062.85 953.841
Preprocessing train data set:  /content/drive/MyDrive/Trajectory_Analysis/Interaction/DR_USA_Roundabout_EP_val.csv
Rotation degree:  0


100%|██████████| 433/433 [00:25<00:00, 17.17it/s]


Preprocessing train data set:  /content/drive/MyDrive/Trajectory_Analysis/Interaction/DR_DEU_Roundabout_OF_val.csv
Rotation degree:  0


100%|██████████| 568/568 [00:11<00:00, 48.05it/s]


Preprocessing train data set:  /content/drive/MyDrive/Trajectory_Analysis/Interaction/DR_CHN_Roundabout_LN_val.csv
Rotation degree:  0


...

100%|██████████| 2429/2429 [01:15<00:00, 32.13it/s]


Preprocessing train data set:  /content/drive/MyDrive/Trajectory_Analysis/Interaction/DR_USA_Roundabout_EP_train.csv
Rotation degree:  0


100%|██████████| 1775/1775 [02:25<00:00, 12.16it/s]


Preprocessing train data set:  /content/drive/MyDrive/Trajectory_Analysis/Interaction/DR_USA_Roundabout_SR_train.csv
Rotation degree:  0


100%|██████████| 1864/1864 [04:31<00:00,  6.86it/s]


In [9]:
loaded_array = np.load(folder + 'preprocessed_data_rot_' + str(360) + '.npy')
print(loaded_array.shape)

(21200, 3, 36, 40)
